# Exercise 4
Write a discrete event simulation program for a blocking system, i.e. a system with m service units and no waiting room. The offered traffic A is the product of the mean arrival rate and the mean service time.

## 1. 
The arrival process is modelled as a Poisson process. Report the fraction of blocked customers, and a confidence interval for this fraction. Choose the service time distribution as exponential. Parameters: m = 10, mean service time = 8 time units, mean time between customers = 1 time unit (corresponding to an offered traffic of 8 Erlang), 10 x 10.000 customers. This system is sufficiently simple such that the analytical solution is known. See the last slide for the solution. Verify your simulation program using this knowledge.

In [270]:
# Known Parameters:
m = 10 # number of service units
mst = 8 # mean service time, in time units
mtbc = 1 # mean time between customers, in time units
batches = 10 # number of batches
customers_per_batch = 10_000 # number of customers per batch

# The analytical solution
A = mst * mtbc  # Offered traffic A
Erlang_B = (A ** m / np.math.factorial(m)) / sum(A ** k / np.math.factorial(k) for k in range(m + 1))

In [280]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm
import scipy.stats as st 

# arrival times are poisson processes
# service times are exponential
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.12173
95% confidence interval: (0.11165537189869987, 0.13180462810130014)


## 2.
The arrival process is modelled as a renewal process using the same parameters as in Part 1 when possible. Report the fraction of blocked customers, and a confidence interval for this fraction for at least the following two cases:

- (a) Experiment with Erlang distributed inter arrival times. The Erlang distribution should have a mean of 1.
- (b) Hyper exponential inter arrival times. The parameters for the hyper exponential distribution should be p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0.

In [302]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 2 (a)

# arrival times are erlang distributed
# service times are exponential
def sim():
    at = np.cumsum(np.random.gamma(1, mtbc, customers_per_batch)) # Erlang arrival times
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.12125999999999999
95% confidence interval: (0.1119487017101631, 0.13057129828983688)


In [298]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 2 (b)

# arrival times have hyper exponential inter arrival times
# service times are exponential

def hyper_exponential(n):
    # Parameters
    p1 = 0.8
    lambda1 = 0.8333
    p2 = 0.2
    lambda2 = 5.0

    # Generate uniform random numbers
    U = np.random.uniform(size=n)
    
    # Determine which exponential distribution to sample from
    exp_distributions = np.where(U < p1, lambda1, lambda2)
    
    return np.random.exponential(scale=1/exp_distributions, size=n)
    

def sim():
    at = np.cumsum(hyper_exponential(customers_per_batch)) # Hyper exponential arrival times
    st = np.random.exponential(scale=mst, size=customers_per_batch) # Exponential service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Mean fraction of blocked customers: 0.13660999999999998
95% confidence interval: (0.12188808348735772, 0.15133191651264224)


## 3.
The arrival process is again a Poisson process like in Part 1. Experiment with different service time distributions with the same mean service time and m as in Part 1 and Part 2.
- (a) Constant service time
- (b) Pareto distributed service times with at least k = 1.05 and k = 2.05.
- (c) Choose one or two other distributions.

In [282]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (a)

# New Parameters:
cst = 8 # constant service time, in time units

# arrival times are poisson processes
# service times are constant
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = cst # Constant service times
    
    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim () for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.11939
95% confidence interval: (0.10908516024044825, 0.12969483975955176)


In [286]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (b)

# New Parameters:
beta = 1 # Just arbitrarily chosen

# arrival times are poisson processes
# service times are pareto distributed
def sim(k):
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = (np.random.pareto(k, customers_per_batch) + beta) * (mst * (k - 1) / k) # Pareto service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac_k_1_05 = [sim(k=1.05) for batch in range(batches)]
blocked_frac_k_2_05 = [sim(k=2.05) for batch in range(batches)]

# k=1.05
# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac_k_1_05)
std_blocked_fraction = np.std(blocked_frac_k_1_05)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Analytical solution (Erlang B formula): {Erlang_B}\n")

print("k=1.05")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {confidence_interval}\n")

# k=2.05
# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac_k_2_05)
std_blocked_fraction = np.std(blocked_frac_k_2_05)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print("k=2.05")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149

k=1.05
Mean fraction of blocked customers: 0.00207
95% confidence interval: (0.12247498484114915, 0.12272501515885086)

k=2.05
Mean fraction of blocked customers: 0.11939999999999999
95% confidence interval: (0.10875583538549219, 0.1300441646145078)


In [296]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson, norm

### 3 (c) - Normally distributed service times

# New Parameters:
mean = 8  # Mean service time
std = 2  # Standard deviation of service time

# arrival times are poisson processes
# service times are pareto distributed
def sim():
    at = np.cumsum(np.random.exponential(scale=mtbc, size=customers_per_batch)) # Poisson arrival times: CDF = P(Xi ≤ t) = 1 − e−λt
    st = np.random.normal(mean, std, customers_per_batch) # Normal service times

    # Event queue for busy service units m
    service_busy_until = np.zeros(m)
    blocked_customers = 0

    # Iterate through each customer
    for i in range(customers_per_batch):
        
        # Check if any service unit is free
        if any(service_busy_until <= at[i]):
            
            # Find the first free service unit
            not_in_use = np.where(service_busy_until <= at[i])[0][0]
            service_busy_until[not_in_use] = at[i] + st[i]
        else:
            blocked_customers += 1
    
    return blocked_customers / customers_per_batch

blocked_frac = [sim() for batch in range(batches)]

# Calculate mean and standard deviation
mean_blocked_fraction = np.mean(blocked_frac)
std_blocked_fraction = np.std(blocked_frac)

# Calculate the 95% confidence interval
ci = st.t.interval(0.95, df=batches-1, loc=mean_blocked_fraction, scale=std_blocked_fraction)

print(f"Analytical solution (Erlang B formula): {Erlang_B}")
print(f"Mean fraction of blocked customers: {mean_blocked_fraction}")
print(f"95% confidence interval: {ci}")

Analytical solution (Erlang B formula): 0.12166106425295149
Mean fraction of blocked customers: 0.12298
95% confidence interval: (0.11476628209885814, 0.13119371790114187)


## 4.  Compare confidence intervals for Parts 1, 2, and 3 then interpret and explain differences if any.
We see that...